Using Pretrained Model for Transfer Learning

In [33]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [34]:
import tensorflow_datasets as tfds
import tensorflow as tf


dataset, info = tfds.load("tf_flowers", split= [
    "train[:10%]", # test set
    "train[10%:25%]", # validation set
    "train[25%:]" # train set
], as_supervised= True, with_info= True)

test_set, valid_set, train_set = dataset
dataset_size = info.splits["train"].num_examples
dataset_size

3670

In [35]:
dataset_classes = info.features["label"].names
dataset_classes

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [36]:
n_classes = info.features["label"].num_classes
n_classes

5

In [37]:
len(train_set)

2752

Preprocessing the Datasets to XCEPTION CNN Architecture

In [38]:
def preprocess(image, label):
    resized_image = tf.image.resize(image, [224, 224])
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

# Tip <br>
add keras.preprocessing.image.ImageDataGenerator() for data augmentation 

In [39]:
# apply preprocess, batch, and prefetch on the dataset
batch_size = 32
train_set = train_set.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [40]:
# load pretrained xception model excluding top layer
base_model = tf.keras.applications.xception.Xception(weights= "imagenet", include_top= False) 
# create custome pooling and dense layer 
avg = tf.keras.layers.AveragePooling2D(pool_size= (2, 2))(base_model.output) # specify pool_size
output = tf.keras.layers.Dense(n_classes, activation= "softmax")(avg)
# create the model 
model = tf.keras.models.Model(inputs= base_model.input, outputs= output)

In [41]:
# freezing pretrained layers at the beginning of training
for layer in base_model.layers:
    layer.trainable = False

In [42]:
model.layers[1].trainable

False

In [ ]:
# compile model
optimizer = tf.keras.optimizers.SGD(learning_rate= 0.2, momentum= 0.9, decay= 0.01)
model.compile(loss= "spare_categorical_crossentropy",
              optimizer= optimizer,
              metrics= ["accuracy"])
history = model.fit(train_set, epochs= 5, validation_data= valid_set)